In [ ]:
import pydicom # библиотека для работы с DICOM файлами

from matplotlib import cm # цветовые схемы для визуализации
from matplotlib import pyplot as plt # библиотека для визуализации 

In [ ]:
import multiprocessing as mp
pool = mp.Pool(mp.cpu_count()-4)
import pydicom # библиотека для работы с DICOM файлами

from matplotlib import cm # цветовые схемы для визуализации
from matplotlib import pyplot as plt # библиотека для визуализации 

import glob
import os.path
from tqdm import tqdm
import cv2

import skimage
from skimage.exposure import equalize_adapthist


from os.path import basename, dirname, splitext
!cd /home/krom/projects/ML
files_path = glob.glob("./**/*.dcm",recursive=True)
files_path
fff= files_path
fff
def dcm2jpg(files):
    #print("files", files, "\n" , len(files))
    #for dcm_name in files:
    # cv.COLORMAP_HSV
    if 1==1:
        dcm_name=files
        imagedata = pydicom.dcmread(dcm_name)
        img = imagedata.pixel_array
        #print(dcm_name + "\n")
        clean_name, exten = splitext(dcm_name)
        p_dir = dirname(dcm_name)
        f_name = basename(dcm_name)
        png_name = '{}.png'.format(clean_name)
        jpg_name = '{}.jpg'.format(clean_name)
        img = skimage.exposure.equalize_adapthist(img)
        #imC = cv2.applyColorMap(im, cv2.COLORMAP_JET)
        #cv2.imwrite(png_name, cv2.applyColorMap(img, cv2.COLORMAP_JET)  )
        !echo $png_name
        
        #plt.imsave(png_name,img,cmap=plt.cm.hsv)
        #!convert $png_name -resize 512x512 -gravity center -background black -extent 512x512 $jpg_name 
        #!rm -f $png_name
        plt.imsave(png_name,img,cmap=plt.cm.bone)
        jpg_name = '{}.bone.jpg'.format(clean_name)
        !convert $png_name -resize 512x512 -gravity center -background black -extent 512x512 $jpg_name 
        !rm -f $png_name $dcm_name
        
    return files

print("hello")

In [ ]:
imagedata = pydicom.dcmread(files_path[733])

f = plt.figure(figsize=(12,12))
ax = f.add_subplot(121)
ax2 = f.add_subplot(122)
img = imagedata.pixel_array
img = skimage.exposure.equalize_adapthist(img)

ax.imshow(imagedata.pixel_array, cmap=plt.cm.bone)
ax2.imshow(img, cmap=plt.cm.hsv)  
plt.imsave("lot.png",imagedata.pixel_array,cmap=plt.cm.hsv)
plt.imsave("cot.png",img,cmap=plt.cm.hsv)
plt.imsave("bot.png",img,cmap=plt.cm.bone)


In [ ]:

pool = mp.Pool(mp.cpu_count())
%time results = pool.map(dcm2jpg, fff)
len(results)

In [ ]:
def seq2jpg(fff):
    for f_f in fff:
        dcm2jpg(f_f)
        
%time seq2jpg(fff)        


# Такой вариант работает, но не учитывает метаданные, которые относятся к изображению. В основном это касается BitsAllocated и PhotometricInterpretation

Значение Photometric Interpretation определяет предполагаемую интерпретацию данных пикселей изображения.

Нас тут интересуют основные два значения для рентгена:

MONOCHROME1 - Пиксельные данные представляют собой одну плоскость монохромного изображения. Минимальное значение выборки предназначено для отображения белым цветом после выполнения любых преобразований шкалы серого VOI.
MONOCHROME2 - Пиксельные данные представляют собой одну плоскость монохромного изображения. Минимальное значение выборки предназначено для отображения черным цветом после выполнения любых преобразований шкалы серого VOI.
Bits Allocated - Количество бит, выделенных для каждой выборки пикселей. Каждой выборке должно быть выделено одинаковое количество битов. Выделенные биты должны быть либо 1, либо кратными 8.

Проверим метатеги у нашего примера и попробуем конвертировать изображение.

print(imagedata.BitsAllocated)
print(imagedata.PhotometricInterpretation)
На нашем примере мы увидим 8 и MONOCHROME2. Код переделывать не придётся. Но в случае если PhotometricInterpretation было бы MONOCHROME1 изображение получилось бы инвертированным.

In [ ]:
#

In [ ]:
print(imagedata.BitsAllocated)
print(imagedata.PhotometricInterpretation)

img = np.invert(imagedata.pixel_array)

В случае если PhotometricInterpretation было бы MONOCHROME1 изображение получилось бы инвертированным. Поэтому используем np.invert, что бы минимальное значение выборки было отображением черного цвета.

Сегментация изображения — это разделение изображения на несколько сегментов (множество пикселей). Сегментация изображений обычно используется для того, чтобы выделить объекты и границы на изображениях. Более точно, сегментация изображений — это процесс присвоения таких меток каждому пикселю изображения, что пиксели с одинаковыми метками имеют общие визуальные характеристики.

Сегментация применяется во многих областях, например, в производстве для индикации дефектов при сборке деталей, в медицине для поиска опухолей и других патологий, также для составления карт местности по снимкам со спутников итд

In [ ]:
#Рассмотрим алгоритм сегментации на основе графов Felsenszwalb из библиотеки skimage. 
#Алгоритм производит сегментацию многоканального (например, RGB) изображения с использованием быстрой кластеризации
# на основе минимального связующего дерева на сетке изображения. 
#Параметр scale задает уровень наблюдения. Чем выше scale, тем меньше сегментов и они большего размера.
# sigma - это диаметр гауссова ядра, используемого для сглаживания изображения перед сегментацией.
#Количество производимых сегментов, а также их размер можно контролировать только косвенно через scale.
# Размер сегмента в изображении может сильно различаться в зависимости от контраста. 
#Для изображений RGB алгоритм использует евклидово расстояние между пикселями в цветовом пространстве.

#Нашей задачей будет в уже знакомом  рентген снимке попробовать сегментировать оба легких.

example = 'drive/MyDrive/siim-acr-pneumothorax-segmentation/stage_2_images/ID_240440107.dcm'
imagedata= pydicom.dcmread(example)

# Импортируем нужные библиотеки
import numpy as np
from skimage.exposure import equalize_hist
from skimage.filters.rank import median
from skimage.morphology import disk
from skimage.segmentation import felzenszwalb
from skimage.transform import rescale

#Занулим значения больше 65

im_thres = imagedata.pixel_array.copy()
im_thres[im_thres > 65] = 0


#Масштабируем и фильтруем и нормализовываем картинку

im_small = rescale(im_thres, 0.65)
im_small_filt = median(im_small, disk(50))   
im_small_filt = equalize_hist(im_small_filt)
#Запускаем сам алгоритм

segments = felzenszwalb(im_small_filt, scale=0.5)
#Визуализируем и посмотрим, что получилось

f = plt.figure(figsize=(12,12))
ax = f.add_subplot(121)
ax2 = f.add_subplot(122)
ax.imshow(imagedata.pixel_array, cmap=plt.cm.bone)
ax2.imshow(segments, cmap=plt.cm.bone)


Надо попробовать поиграть с параметрами, пока получается довольно-таки хреново.Лучше всего с сегментацией справляются нейронные сети

Для начала рассмотрим библиотеку lungs-segmentation. Она как раз решает задачу сегментации правого и левого лёгкого.


In [ ]:
pip install lungs-segmentation

In [ ]:
#Библиотека работает с PNG изображениями, поэтому сначала сконвертируем изображение уже знакомым нам способом.
from skimage.transform import resize
import cv2

example = 'drive/MyDrive/siim-acr-pneumothorax-segmentation/stage_2_images/ID_240440107.dcm'
imagedata= pydicom.dcmread(example)

img =imagedata.pixel_array
name = example.split('/')[-1][:-4]
img = resize(img,(512,512))
cv2.imwrite('{}.png'.format(name), img * 255)

#После чего импортируем lungs-segmentation и запускаем обработку моделью

from lungs_segmentation.pre_trained_models import create_model
import lungs_segmentation.inference as inference
import torch

model = create_model("resnet34")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

plt.figure(figsize=(12,12))
plt.subplot(1,1,1)
image, mask = inference.inference(model,'ID_240440107.png', 0.2)
plt.imshow(inference.img_with_masks( image, [mask[0], mask[1]], alpha = 0.1))